In [3]:
from sklearn import metrics
from scipy.stats import t as the
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn import tree
from sklearn.neighbors import kneighbors_graph
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 
import networkx as nx
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
from sklearn import cluster, datasets, mixture
import time
from sklearn import manifold
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community.centrality import girvan_newman
import itertools
from networkx.algorithms.community import label_propagation_communities
import community
from sklearn.metrics.cluster import normalized_mutual_info_score
import pickle as pkl
import pandas as pd
import numpy as np
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community.centrality import girvan_newman
import itertools
from networkx.algorithms.community import label_propagation_communities
import community
from sklearn.metrics.cluster import normalized_mutual_info_score
import math
import sklearn
from sklearn import preprocessing
from sklearn.decomposition import PCA
import time
from sklearn.manifold import TSNE
import numpy as np 
import seaborn as sns
import umap

In [5]:
for unit in ['acc','gyr','mag','LA','LL','RA','RL','T']: 
    dbfile = open('datasets/das/'+unit+'_fe', 'rb') 
    data = pkl.load(dbfile) 
    train, validate, test = pd.DataFrame(data[0]), pd.DataFrame(data[1]), pd.DataFrame(data[2]) 
    data_x, data_y = train.iloc[:,2:], train.iloc[:,0] 
    nmi_all = []
    v_all = []
    nmi = []
    for i in range(1,9):
        sub1_x = np.array(data_x[train.iloc[:,1]==i])
        Y_all = np.array(data_y[train.iloc[:,1]==i]).astype(int)
        X_all = StandardScaler().fit_transform(sub1_x)


        X_reduced = umap.UMAP().fit_transform(X_all)
        X = X_reduced
        Y = Y_all
        
        k = int(2*math.log(len(X),2))
        A = kneighbors_graph(X, n_neighbors=k, mode='distance', include_self=False, metric='euclidean')
        A = A.toarray()
        maxvalue = np.max(A)
        A = A/maxvalue
        G = nx.Graph(A)

        distance = np.array(A)


        for i in range(len(A)):
            for j in range(len(A)):
                if A[i][j] != A[j][i]:
                    A[i][j] = A[j][i] = 0
                    distance[i][j] = distance[j][i] = 0
                    G.remove_edge(i,j)
                elif A[i][j]:
                    G.add_edge(i,j,weight= 1/A[i][j])
                    distance[i][j] = 1/A[i][j]
        G.graph['edge_weight_attr'] = 'weight'
        
        G.remove_nodes_from(list(nx.isolates(G)))
        print(list(nx.isolate))

        lp = list(label_propagation_communities(G))
        lpcoms = [tuple(x) for x in lp]
        y_lp = np.zeros((len(G.nodes))).astype(int)
        nodes = list(G.nodes)
        for i in range(len(lpcoms)):
            com = list(lpcoms[i])
            for node in com:
                y_lp[nodes.index(node)]= i
        parts = y_lp        
        part_len= len(set(parts))

        l =[]
        size = []
        purity = np.zeros((part_len))
        for j in range(part_len):
            part = [indx for indx, n in enumerate(parts) if n==j]
            size.append(len(part))
            vote = np.zeros(max(Y)+1)
            for p in part:
                vote[Y[p]-1] += 1
            purity[j] = vote.max()/sum(vote)
            l.append(np.argmax(vote))

        #print(np.mean(purity))
        #print(purity)
        #print(part_len)
        #print(l)
        #print(size)

        edges1 =  np.nonzero(A)[0]
        edges2 =  np.nonzero(A)[1]

        w_density = np.zeros((part_len,part_len))
        density = np.zeros((part_len,part_len))

        for e in range(len(edges1)):
            w_density[parts[edges1[e]]][parts[edges2[e]]] += distance[edges1[e]][edges2[e]]
            w_density[parts[edges2[e]]][parts[edges1[e]]] += distance[edges2[e]][edges1[e]]
            density[parts[edges1[e]]][parts[edges2[e]]] += 1
            density[parts[edges2[e]]][parts[edges1[e]]] += 1

        for e in range(part_len):
            w_density[e][e] = w_density[e][e]/2
            density[e][e] = density[e][e]/2

        def similarity(w_density,density,i,j,m=10**3,beta=1,alpha=2):

            avg_weight_i = w_density[i][i]/density[i][i]
            avg_weight_j = w_density[j][j]/density[j][j]
            p1 = min(avg_weight_i,avg_weight_j)
            p2 = max(avg_weight_i,avg_weight_j)
            if p2==0:
                p = 1
            else:
                p = p1/p2

            if not w_density[i][j]:
                avg_weight_ij = 0
            else:    
                avg_weight_ij = w_density[i][j]/density[i][j]

            if w_density[i][i]+w_density[j][j] == 0:
                closeness = m*avg_weight_ij
            else:
                closeness = m*(avg_weight_ij/(w_density[i][i]+w_density[j][j]))
            if (density[i][i]+density[j][j])/2 == 0:
                inter_connectivity = (p**beta)*(density[i][j])
            else:
                inter_connectivity = (p**beta)*(density[i][j]/((density[i][i]+density[j][j])/2))    

            return (closeness**alpha)*inter_connectivity



        sim_matrix = np.zeros((part_len,part_len))

        for i in range(part_len):
            for j in range(part_len):
                if i!=j:
                    sim_matrix[j][i]= sim_matrix[i][j]= similarity(w_density,density,i,j)
                else:
                    sim_matrix[i][j]=0

        sim_matrix1 = np.array(sim_matrix)
        sim_matrix1[np.isnan(sim_matrix1)] = 0
        clusters = np.array(parts)
        iter = 0
        w_density1 = np.array(w_density)
        density1 = np.array(density)
        thresh = np.nanmedian(sim_matrix1[np.nonzero(sim_matrix1)])/2
        while((np.max(sim_matrix1)>thresh) & (len(set(clusters))>12)):
            iter += 1
            result = np.where(sim_matrix1 == np.amax(sim_matrix1))
            if result[0][0]<result[1][0]:  
                i1 = result[0][0]
                i2 = result[1][0]
            else:
                i1 = result[1][0]
                i2 = result[0][0]

            #print('*****iteration',iter,'***********')

            #print('clusters:',i1,i2,'labels:',l[i1],l[i2],'distance:', np.amax(sim_matrix1))

            clusters[clusters==i2]=i1
            clusters[clusters>i2] -= 1

            cluster_len = len(set(clusters))
            l =np.zeros((cluster_len))
            purity = np.zeros((cluster_len))
            for k in range(cluster_len):
                part = [indx for indx, n in enumerate(clusters) if n==k]
                vote = []
                vote = np.zeros(max(Y)+1)
                for p in part:
                    vote[Y[p]-1] += 1
                purity[k] = vote.max()/sum(vote)
                l[k] = np.argmax(vote)

            #print('purity:',np.mean(purity), 'cluster#:', len(set(clusters)))
            #print('nmi:', normalized_mutual_info_score(clusters,Y))


            #print('*********************************')

            for i in range(len(w_density1)):
                w_density1[i][i1] = w_density1[i][i1] + w_density1[i][i2]
                w_density1[i1][i] = w_density1[i][i1]
                density1[i][i1] = density1[i][i1] + density1[i][i2]
                density1[i1][i] = density1[i][i1]

            for j in range(len(sim_matrix1)):
                if i1!=j:
                    sim_matrix1[i1][j]= similarity(w_density1,density1,i1,j)
                    #print( sim_matrix1[i1][j],i1,j)
                else:
                    sim_matrix1[i1][j]=0
                    sim_matrix1[j][i1]=0

            w_density1=np.delete(w_density1,i2,0)
            w_density1=np.delete(w_density1,i2,1)

            density1=np.delete(density1,i2,0)
            density1=np.delete(density1,i2,1)

            sim_matrix1=np.delete(sim_matrix1,i2,0)
            sim_matrix1=np.delete(sim_matrix1,i2,1)

        #print('purity:',np.mean(purity), 'cluster#:', len(set(clusters)))
        #print('nmi:', normalized_mutual_info_score(clusters,Y))

        nmi.append(normalized_mutual_info_score(clusters,Y))

    avg_nmi = np.mean(np.array(nmi),axis=0)
    std_nmi = np.std(np.array(nmi),axis=0)
    print('**********'+unit+'*************')
    print(avg_nmi)

    dbfile = open('datasets/pamap2/'+unit+'_avg_nmi_score_cci', 'ab') 
    pkl.dump(avg_nmi,dbfile)
    dbfile.close()

    dbfile = open('datasets/pamap2/'+unit+'_std_nmi_score_cci', 'ab') 
    pkl.dump(std_nmi,dbfile)
    dbfile.close()

/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of '

**********acc*************
0.8741839042240289


/home/paris/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: RuntimeWarning: invalid value encountered in double_scalars
/home/paris/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:88: RuntimeWarning: invalid value encountered in double_scalars
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of

**********gyr*************
0.7934246070196987


/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of '

...

/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of '

**********RL*************
0.8670252181666903


/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: RuntimeWarning: invalid value encountered in double_scalars
/home/paris/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:88: RuntimeWarning: invalid value encountered in double_scalars
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of

**********T*************
0.8723317975859312


/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


In [6]:
for unit in ['acc','v']: 
    dbfile = open('datasets/smartsocks/'+unit+'_fe', 'rb') 
    data = pkl.load(dbfile) 
    train, validate, test = pd.DataFrame(data[0]), pd.DataFrame(data[1]), pd.DataFrame(data[2]) 
    data_x, data_y = train.iloc[:,0:-2], train.iloc[:,-2] #smartsocks

    nmi_all = []
    v_all = []
    ari_all = []
    nmi = []
    for i in [1,2,3,4,5,6,7,9,10,11,12,14]:
        sub1_x = np.array(data_x[train.iloc[:,-1]==i]).astype(np.float64)
        Y_all = np.array(data_y[train.iloc[:,-1]==i]).astype(int)
        X_all = StandardScaler().fit_transform(sub1_x)


        tsne = manifold.TSNE(n_components=2, init='random',random_state=0, perplexity=20)
        X_reduced = tsne.fit_transform(X_all)
        X = X_reduced
        #X = X_all
        Y = Y_all
        
        k = int(2*math.log(len(X),2))
        A = kneighbors_graph(X, n_neighbors=k, mode='distance', include_self=False, metric='euclidean')
        A = A.toarray()
        maxvalue = np.max(A)
        A = A/maxvalue
        G = nx.Graph(A)

        distance = np.array(A)


        for i in range(len(A)):
            for j in range(len(A)):
                if A[i][j] != A[j][i]:
                    A[i][j] = A[j][i] = 0
                    distance[i][j] = distance[j][i] = 0
                    G.remove_edge(i,j)
                elif A[i][j]:
                    G.add_edge(i,j,weight= 1/A[i][j])
                    distance[i][j] = 1/A[i][j]
        G.graph['edge_weight_attr'] = 'weight'

        lp = list(label_propagation_communities(G))
        lpcoms = [tuple(x) for x in lp]
        y_lp = np.zeros((len(G.nodes))).astype(int)
        nodes = list(G.nodes)
        for i in range(len(lpcoms)):
            com = list(lpcoms[i])
            for node in com:
                y_lp[nodes.index(node)]= i
        parts = y_lp        
        part_len= len(set(parts))

        l =[]
        size = []
        purity = np.zeros((part_len))
        for j in range(part_len):
            part = [indx for indx, n in enumerate(parts) if n==j]
            size.append(len(part))
            vote = np.zeros(max(Y)+1)
            for p in part:
                vote[Y[p]-1] += 1
            purity[j] = vote.max()/sum(vote)
            l.append(np.argmax(vote))

        #print(np.mean(purity))
        #print(purity)
        #print(part_len)
        #print(l)
        #print(size)

        edges1 =  np.nonzero(A)[0]
        edges2 =  np.nonzero(A)[1]

        w_density = np.zeros((part_len,part_len))
        density = np.zeros((part_len,part_len))

        for e in range(len(edges1)):
            w_density[parts[edges1[e]]][parts[edges2[e]]] += distance[edges1[e]][edges2[e]]
            w_density[parts[edges2[e]]][parts[edges1[e]]] += distance[edges2[e]][edges1[e]]
            density[parts[edges1[e]]][parts[edges2[e]]] += 1
            density[parts[edges2[e]]][parts[edges1[e]]] += 1

        for e in range(part_len):
            w_density[e][e] = w_density[e][e]/2
            density[e][e] = density[e][e]/2

        def similarity(w_density,density,i,j,m=10**3,beta=1,alpha=2):

            avg_weight_i = w_density[i][i]/density[i][i]
            avg_weight_j = w_density[j][j]/density[j][j]
            p1 = min(avg_weight_i,avg_weight_j)
            p2 = max(avg_weight_i,avg_weight_j)
            if p2==0:
                p = 1
            else:
                p = p1/p2

            if not w_density[i][j]:
                avg_weight_ij = 0
            else:    
                avg_weight_ij = w_density[i][j]/density[i][j]

            if w_density[i][i]+w_density[j][j] == 0:
                closeness = m*avg_weight_ij
            else:
                closeness = m*(avg_weight_ij/(w_density[i][i]+w_density[j][j]))
            if (density[i][i]+density[j][j])/2 == 0:
                inter_connectivity = (p**beta)*(density[i][j])
            else:
                inter_connectivity = (p**beta)*(density[i][j]/((density[i][i]+density[j][j])/2))    

            return (closeness**alpha)*inter_connectivity



        sim_matrix = np.zeros((part_len,part_len))

        for i in range(part_len):
            for j in range(part_len):
                if i!=j:
                    sim_matrix[j][i]= sim_matrix[i][j]= similarity(w_density,density,i,j)
                else:
                    sim_matrix[i][j]=0

        sim_matrix1 = np.array(sim_matrix)
        sim_matrix1[np.isnan(sim_matrix1)] = 0
        clusters = np.array(parts)
        iter = 0
        w_density1 = np.array(w_density)
        density1 = np.array(density)
        thresh = np.nanmedian(sim_matrix1[np.nonzero(sim_matrix1)])/2
        while((np.max(sim_matrix1)>thresh) & (len(set(clusters))>12)):
            iter += 1
            result = np.where(sim_matrix1 == np.amax(sim_matrix1))
            if result[0][0]<result[1][0]:  
                i1 = result[0][0]
                i2 = result[1][0]
            else:
                i1 = result[1][0]
                i2 = result[0][0]

            #print('*****iteration',iter,'***********')

            #print('clusters:',i1,i2,'labels:',l[i1],l[i2],'distance:', np.amax(sim_matrix1))

            clusters[clusters==i2]=i1
            clusters[clusters>i2] -= 1

            cluster_len = len(set(clusters))
            l =np.zeros((cluster_len))
            purity = np.zeros((cluster_len))
            for k in range(cluster_len):
                part = [indx for indx, n in enumerate(clusters) if n==k]
                vote = []
                vote = np.zeros(max(Y)+1)
                for p in part:
                    vote[Y[p]-1] += 1
                purity[k] = vote.max()/sum(vote)
                l[k] = np.argmax(vote)

            #print('purity:',np.mean(purity), 'cluster#:', len(set(clusters)))
            #print('nmi:', normalized_mutual_info_score(clusters,Y))


            #print('*********************************')

            for i in range(len(w_density1)):
                w_density1[i][i1] = w_density1[i][i1] + w_density1[i][i2]
                w_density1[i1][i] = w_density1[i][i1]
                density1[i][i1] = density1[i][i1] + density1[i][i2]
                density1[i1][i] = density1[i][i1]

            for j in range(len(sim_matrix1)):
                if i1!=j:
                    sim_matrix1[i1][j]= similarity(w_density1,density1,i1,j)
                    #print( sim_matrix1[i1][j],i1,j)
                else:
                    sim_matrix1[i1][j]=0
                    sim_matrix1[j][i1]=0

            w_density1=np.delete(w_density1,i2,0)
            w_density1=np.delete(w_density1,i2,1)

            density1=np.delete(density1,i2,0)
            density1=np.delete(density1,i2,1)

            sim_matrix1=np.delete(sim_matrix1,i2,0)
            sim_matrix1=np.delete(sim_matrix1,i2,1)

        #print('purity:',np.mean(purity), 'cluster#:', len(set(clusters)))
        #print('nmi:', normalized_mutual_info_score(clusters,Y))

        nmi.append(normalized_mutual_info_score(clusters,Y))

    avg_nmi = np.mean(np.array(nmi),axis=0)
    std_nmi = np.std(np.array(nmi),axis=0)
    print('**********'+unit+'*************')
    print(avg_nmi)

    dbfile = open('datasets/pamap2/'+unit+'_avg_nmi_score_cci', 'ab') 
    pkl.dump(avg_nmi,dbfile)
    dbfile.close()

    dbfile = open('datasets/pamap2/'+unit+'_std_nmi_score_cci', 'ab') 
    pkl.dump(std_nmi,dbfile)
    dbfile.close()

/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of '

**********acc*************
0.8398001812358958


/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of '

**********v*************
0.7359849135325743


/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


In [23]:
units = ['acc','gyr','mag','ori','tmp','hr','chest','ankle','hand']
units = ['gyr']
for unit in units:
    dbfile = open('/home/paris/datasets/pamap2/'+unit+'_fe', 'rb') 
    data = pkl.load(dbfile) 
    train, validate, test = pd.DataFrame(data[0]), pd.DataFrame(data[1]), pd.DataFrame(data[2]) 
    data_x, data_y = train.iloc[:,2:], train.iloc[:,0] #pamap2

    nmi = []
    pur = []
    for sub in [1,2,3,4,5,6,7,8]:
        max_nmi = 0
        purity_max = 0
        sub1_x = np.array(data_x[train.iloc[:,1]==sub])
        Y_all = np.array(data_y[train.iloc[:,1]==sub]).astype(int)
        X_all = StandardScaler().fit_transform(sub1_x)


        tsne = manifold.TSNE(n_components=2, init='random',random_state=0, perplexity=20)
        X_reduced = tsne.fit_transform(X_all)
        X = X_reduced
        #X = X_all
        Y = Y_all

        Y = Y_all
        Y[Y==12]=8
        Y[Y==13]=9
        Y[Y==16]=10
        Y[Y==17]=11
        Y[Y==24]=12

        k = int(2*math.log(len(X),2))
        A = kneighbors_graph(X, n_neighbors=k, mode='distance', include_self=False, metric='euclidean')
        A = A.toarray()
        maxvalue = np.max(A)
        #A = A/maxvalue
        G = nx.Graph(A)

        distance = np.array(A)


        for i in range(len(A)):
            for j in range(len(A)):
                if A[i][j] != A[j][i]:
                    A[i][j] = A[j][i] = 0
                    distance[i][j] = distance[j][i] = 0
                    G.remove_edge(i,j)
                elif A[i][j]:
                    G.add_edge(i,j,weight= maxvalue-A[i][j])
                    distance[i][j] = maxvalue-A[i][j]
        G.graph['edge_weight_attr'] = 'weight'

        lp = list(label_propagation_communities(G))
        lpcoms = [tuple(x) for x in lp]
        y_lp = np.zeros((len(G.nodes))).astype(int)
        nodes = list(G.nodes)
        for i in range(len(lpcoms)):
            com = list(lpcoms[i])
            for node in com:
                y_lp[nodes.index(node)]= i
        parts = y_lp        
        part_len= len(set(parts))
        

        l =[]
        size = []
        purity = np.zeros((part_len))
        for j in range(part_len):
            part = [indx for indx, n in enumerate(parts) if n==j]
            size.append(len(part))
            vote = np.zeros(max(Y)+1)
            for p in part:
                vote[Y[p]-1] += 1
            purity[j] = vote.max()/sum(vote)
            l.append(np.argmax(vote))


        edges1 =  np.nonzero(A)[0]
        edges2 =  np.nonzero(A)[1]

        w_density = np.zeros((part_len,part_len))
        density = np.zeros((part_len,part_len))

        for e in range(len(edges1)):
            w_density[parts[edges1[e]]][parts[edges2[e]]] += distance[edges1[e]][edges2[e]]
            w_density[parts[edges2[e]]][parts[edges1[e]]] += distance[edges2[e]][edges1[e]]
            density[parts[edges1[e]]][parts[edges2[e]]] += 1
            density[parts[edges2[e]]][parts[edges1[e]]] += 1

        for e in range(part_len):
            w_density[e][e] = w_density[e][e]/2
            density[e][e] = density[e][e]/2

        def similarity(w_density,density,i,j,m=10**3,beta=1,alpha=2):

            avg_weight_i = w_density[i][i]/density[i][i]
            avg_weight_j = w_density[j][j]/density[j][j]
            p1 = min(avg_weight_i,avg_weight_j)
            p2 = max(avg_weight_i,avg_weight_j)
            if p2==0:
                p = 1
            else:
                p = p1/p2

            if not w_density[i][j]:
                avg_weight_ij = 0
            else:    
                avg_weight_ij = w_density[i][j]/density[i][j]

            if w_density[i][i]+w_density[j][j] == 0:
                closeness = m*avg_weight_ij
            else:
                closeness = m*(avg_weight_ij/(w_density[i][i]+w_density[j][j]))
            if (density[i][i]+density[j][j])/2 == 0:
                inter_connectivity = (p**beta)*(density[i][j])
            else:
                inter_connectivity = (p**beta)*(density[i][j]/((density[i][i]+density[j][j])/2))    

            return (closeness**alpha)*inter_connectivity
        
        
        def similarity1(w_density,density,i,j,m=10**3,beta=1,alpha=2,parts):
            
            p1 = density[i][j]/
            p2 = (density[i][i]+density[j][j])/2
            if p2:
                return p1/p2
            else:
                return p1



        sim_matrix = np.zeros((part_len,part_len))

        for i in range(part_len):
            for j in range(part_len):
                if i!=j:
                    sim_matrix[j][i]= sim_matrix[i][j]= similarity1(w_density,density,i,j)
                else:
                    sim_matrix[i][j]=0

        sim_matrix1 = np.array(sim_matrix)
        sim_matrix1[np.isnan(sim_matrix1)] = 0
        clusters = np.array(parts)
        iter = 0
        w_density1 = np.array(w_density)
        density1 = np.array(density)
        thresh = 0#np.nanmedian(sim_matrix1[np.nonzero(sim_matrix1)])/2
        while((np.max(sim_matrix1)>thresh) & (len(set(clusters))>12)):
            iter += 1
            result = np.where(sim_matrix1 == np.amax(sim_matrix1))
            if result[0][0]<result[1][0]:  
                i1 = result[0][0]
                i2 = result[1][0]
            else:
                i1 = result[1][0]
                i2 = result[0][0]

            print('*****iteration',iter,'***********')

            print('clusters:',i1,i2,'labels:',l[i1],l[i2],'distance:', np.amax(sim_matrix1))

            clusters[clusters==i2]=i1
            clusters[clusters>i2] -= 1

            cluster_len = len(set(clusters))
            l =np.zeros((cluster_len))
            purity = np.zeros((cluster_len))
            for k in range(cluster_len):
                part = [indx for indx, n in enumerate(clusters) if n==k]
                vote = []
                vote = np.zeros(max(Y)+1)
                for p in part:
                    vote[Y[p]-1] += 1
                purity[k] = vote.max()/sum(vote)
                l[k] = np.argmax(vote)

            #print('purity:',np.mean(purity), 'cluster#:', len(set(clusters)))
            #print('nmi:', normalized_mutual_info_score(clusters,Y))
            if max_nmi<normalized_mutual_info_score(clusters,Y):
                max_purity = np.mean(purity)
                max_nmi=normalized_mutual_info_score(clusters,Y)

            print('*********************************')

            for i in range(len(w_density1)):
                w_density1[i][i1] = w_density1[i][i1] + w_density1[i][i2]
                w_density1[i1][i] = w_density1[i][i1]
                density1[i][i1] = density1[i][i1] + density1[i][i2]
                density1[i1][i] = density1[i][i1]

            for j in range(len(sim_matrix1)):
                if i1!=j:
                    sim_matrix1[i1][j]= similarity1(w_density1,density1,i1,j)
                    #print( sim_matrix1[i1][j],i1,j)
                else:
                    sim_matrix1[i1][j]=0
                    sim_matrix1[j][i1]=0

            w_density1=np.delete(w_density1,i2,0)
            w_density1=np.delete(w_density1,i2,1)

            density1=np.delete(density1,i2,0)
            density1=np.delete(density1,i2,1)

            sim_matrix1=np.delete(sim_matrix1,i2,0)
            sim_matrix1=np.delete(sim_matrix1,i2,1)

        #print('purity:',np.mean(purity), 'cluster#:', len(set(clusters)))
        #print('nmi:', normalized_mutual_info_score(clusters,Y))

        #nmi.append(normalized_mutual_info_score(clusters,Y))
        nmi.append(max_nmi)
        pur.append(purity_max)

    avg_nmi = np.mean(np.array(nmi),axis=0)
    std_nmi = np.std(np.array(nmi),axis=0)
    avg_purity = np.mean(np.array(pur),axis=0)
    std_purity = np.std(np.array(pur),axis=0)
    print('**********'+unit+'*************')
    print(avg_nmi)
    print(avg_purity)

    dbfile = open('datasets/pamap2/'+unit+'_avg_nmi_score_cci', 'ab') 
    pkl.dump(avg_nmi,dbfile)
    dbfile.close()

    dbfile = open('datasets/pamap2/'+unit+'_std_nmi_score_cci', 'ab') 
    pkl.dump(std_nmi,dbfile)
    dbfile.close()

*****iteration 1 ***********
clusters: 30 51 labels: 0 1 distance: 0.5280898876404494
*********************************
*****iteration 2 ***********
clusters: 27 43 labels: 11.0 11.0 distance: 0.5139664804469274
*********************************
*****iteration 3 ***********
clusters: 14 49 labels: 10.0 10.0 distance: 0.4090909090909091


/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


*********************************
*****iteration 4 ***********
clusters: 10 50 labels: 9.0 9.0 distance: 0.39759036144578314
*********************************
*****iteration 5 ***********
clusters: 13 51 labels: 0.0 0.0 distance: 0.38405797101449274
*********************************
*****iteration 6 ***********
clusters: 34 50 labels: 4.0 4.0 distance: 0.3208955223880597
*********************************
*****iteration 7 ***********
clusters: 26 36 labels: 5.0 5.0 distance: 0.29411764705882354
*********************************
*****iteration 8 ***********
clusters: 30 33 labels: 1.0 0.0 distance: 0.291220556745182
*********************************
*****iteration 9 ***********
clusters: 28 41 labels: 3.0 6.0 distance: 0.2805194805194805
*********************************
*****iteration 10 ***********
clusters: 6 41 labels: 4.0 4.0 distance: 0.2796352583586626
*********************************
*****iteration 11 ***********
clusters: 13 30 labels: 0.0 0.0 distance: 0.2660098522167488
*****

/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


*********************************
*****iteration 3 ***********
clusters: 6 16 labels: 11.0 11.0 distance: 0.3368421052631579
*********************************
*****iteration 4 ***********
clusters: 6 48 labels: 11.0 11.0 distance: 0.3688760806916426
*********************************
*****iteration 5 ***********
clusters: 10 30 labels: 10.0 10.0 distance: 0.33217993079584773
*********************************
*****iteration 6 ***********
clusters: 26 32 labels: 9.0 9.0 distance: 0.301255230125523
*********************************
*****iteration 7 ***********
clusters: 4 46 labels: 5.0 5.0 distance: 0.3010752688172043
*********************************
*****iteration 8 ***********
clusters: 1 18 labels: 7.0 7.0 distance: 0.2961165048543689
*********************************
*****iteration 9 ***********
clusters: 25 34 labels: 9.0 9.0 distance: 0.28923076923076924
*********************************
*****iteration 10 ***********
clusters: 18 29 labels: 0.0 2.0 distance: 0.28717948717948716
***

...

*********************************
*****iteration 3 ***********
clusters: 5 25 labels: 10.0 10.0 distance: 0.4161849710982659
*********************************
*****iteration 4 ***********
clusters: 5 26 labels: 10.0 10.0 distance: 0.3937007874015748
*********************************
*****iteration 5 ***********
clusters: 27 33 labels: 8.0 8.0 distance: 0.37209302325581395
*********************************
*****iteration 6 ***********
clusters: 23 29 labels: 1.0 1.0 distance: 0.358974358974359
*********************************
*****iteration 7 ***********
clusters: 0 3 labels: 9.0 9.0 distance: 0.35036496350364965
*********************************
*****iteration 8 ***********
clusters: 6 36 labels: 0.0 0.0 distance: 0.3460207612456747
*********************************
*****iteration 9 ***********
clusters: 14 26 labels: 8.0 8.0 distance: 0.3263157894736842
*********************************
*****iteration 10 ***********
clusters: 18 19 labels: 3.0 3.0 distance: 0.3179916317991632
*******

/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


*********************************
*****iteration 4 ***********
clusters: 3 13 labels: 10.0 10.0 distance: 0.32065217391304346
*********************************
*****iteration 5 ***********
clusters: 3 29 labels: 10.0 10.0 distance: 0.3508771929824561
*********************************
*****iteration 6 ***********
clusters: 26 30 labels: 6.0 6.0 distance: 0.3023255813953488
*********************************
*****iteration 7 ***********
clusters: 7 34 labels: 7.0 7.0 distance: 0.2878787878787879
*********************************
*****iteration 8 ***********
clusters: 24 27 labels: 3.0 3.0 distance: 0.28019323671497587
*********************************
*****iteration 9 ***********
clusters: 14 15 labels: 3.0 3.0 distance: 0.272
*********************************
*****iteration 10 ***********
clusters: 21 23 labels: 3.0 3.0 distance: 0.27104722792607805
*********************************
*****iteration 11 ***********
clusters: 10 25 labels: 0.0 0.0 distance: 0.27058823529411763
**************

/home/paris/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


*********************************
*****iteration 3 ***********
clusters: 12 43 labels: 6.0 6.0 distance: 0.5240641711229946
*********************************
*****iteration 4 ***********
clusters: 34 52 labels: 3.0 3.0 distance: 0.5116279069767442
*********************************
*****iteration 5 ***********
clusters: 2 10 labels: 4.0 4.0 distance: 0.5
*********************************
*****iteration 6 ***********
clusters: 9 57 labels: 5.0 5.0 distance: 0.3983402489626556
*********************************
*****iteration 7 ***********
clusters: 2 4 labels: 4.0 4.0 distance: 0.39543726235741444
*********************************
*****iteration 8 ***********
clusters: 13 36 labels: 0.0 2.0 distance: 0.38613861386138615
*********************************
*****iteration 9 ***********
clusters: 2 23 labels: 4.0 4.0 distance: 0.36633663366336633
*********************************
*****iteration 10 ***********
clusters: 31 40 labels: 3.0 3.0 distance: 0.3573667711598746
************************

In [ ]:
units = ['acc','gyr','mag','ori','tmp','hr','chest','ankle','hand']
for unit in units:
    dbfile = open('/home/paris/datasets/pamap2/'+unit+'_fe', 'rb') 
    data = pkl.load(dbfile) 
    train, validate, test = pd.DataFrame(data[0]), pd.DataFrame(data[1]), pd.DataFrame(data[2]) 
    data_x, data_y = train.iloc[:,2:], train.iloc[:,0] #pamap2

    nmi = []
    for sub in [1,2,3,4,5,6,7,8]:
        sub1_x = np.array(data_x[train.iloc[:,1]==sub])
        Y_all = np.array(data_y[train.iloc[:,1]==sub]).astype(int)
        X_all = StandardScaler().fit_transform(sub1_x)


        tsne = manifold.TSNE(n_components=2, init='random',random_state=0, perplexity=20)
        X_reduced = tsne.fit_transform(X_all)
        X = X_reduced
        #X = X_all
        Y = Y_all

        Y = Y_all
        Y[Y==12]=8
        Y[Y==13]=9
        Y[Y==16]=10
        Y[Y==17]=11
        Y[Y==24]=12

        k = int(2*math.log(len(X),2))
        A = kneighbors_graph(X, n_neighbors=k, mode='distance', include_self=False, metric='euclidean')
        A = A.toarray()
        maxvalue = np.max(A)
        A = A/maxvalue
        G = nx.Graph(A)

        distance = np.array(A)


        for i in range(len(A)):
            for j in range(len(A)):
                if A[i][j] != A[j][i]:
                    A[i][j] = A[j][i] = 0
                    distance[i][j] = distance[j][i] = 0
                    G.remove_edge(i,j)
                elif A[i][j]:
                    G.add_edge(i,j,weight= 1/A[i][j])
                    distance[i][j] = 1/A[i][j]
        G.graph['edge_weight_attr'] = 'weight'

        lp = list(label_propagation_communities(G))
        lpcoms = [tuple(x) for x in lp]
        y_lp = np.zeros((len(G.nodes))).astype(int)
        nodes = list(G.nodes)
        for i in range(len(lpcoms)):
            com = list(lpcoms[i])
            for node in com:
                y_lp[nodes.index(node)]= i
        parts = y_lp        
        part_len= len(set(parts))

        l =[]
        size = []
        purity = np.zeros((part_len))
        for j in range(part_len):
            part = [indx for indx, n in enumerate(parts) if n==j]
            size.append(len(part))
            vote = np.zeros(max(Y)+1)
            for p in part:
                vote[Y[p]-1] += 1
            purity[j] = vote.max()/sum(vote)
            l.append(np.argmax(vote))

        #print(np.mean(purity))
        #print(purity)
        #print(part_len)
        #print(l)
        #print(size)

        edges1 =  np.nonzero(A)[0]
        edges2 =  np.nonzero(A)[1]

        w_density = np.zeros((part_len,part_len))
        density = np.zeros((part_len,part_len))

        for e in range(len(edges1)):
            w_density[parts[edges1[e]]][parts[edges2[e]]] += distance[edges1[e]][edges2[e]]
            w_density[parts[edges2[e]]][parts[edges1[e]]] += distance[edges2[e]][edges1[e]]
            density[parts[edges1[e]]][parts[edges2[e]]] += 1
            density[parts[edges2[e]]][parts[edges1[e]]] += 1

        for e in range(part_len):
            w_density[e][e] = w_density[e][e]/2
            density[e][e] = density[e][e]/2

        def similarity(w_density,density,i,j,m=10**3,beta=1,alpha=2):

            avg_weight_i = w_density[i][i]/density[i][i]
            avg_weight_j = w_density[j][j]/density[j][j]
            p1 = min(avg_weight_i,avg_weight_j)
            p2 = max(avg_weight_i,avg_weight_j)
            if p2==0:
                p = 1
            else:
                p = p1/p2

            if not w_density[i][j]:
                avg_weight_ij = 0
            else:    
                avg_weight_ij = w_density[i][j]/density[i][j]

            if w_density[i][i]+w_density[j][j] == 0:
                closeness = m*avg_weight_ij
            else:
                closeness = m*(avg_weight_ij/(w_density[i][i]+w_density[j][j]))
            if (density[i][i]+density[j][j])/2 == 0:
                inter_connectivity = (p**beta)*(density[i][j])
            else:
                inter_connectivity = (p**beta)*(density[i][j]/((density[i][i]+density[j][j])/2))    

            return (closeness**alpha)*inter_connectivity



        sim_matrix = np.zeros((part_len,part_len))

        for i in range(part_len):
            for j in range(part_len):
                if i!=j:
                    sim_matrix[j][i]= sim_matrix[i][j]= similarity(w_density,density,i,j)
                else:
                    sim_matrix[i][j]=0

        sim_matrix1 = np.array(sim_matrix)
        sim_matrix1[np.isnan(sim_matrix1)] = 0
        clusters = np.array(parts)
        iter = 0
        w_density1 = np.array(w_density)
        density1 = np.array(density)
        thresh = np.nanmedian(sim_matrix1[np.nonzero(sim_matrix1)])/2
        while((np.max(sim_matrix1)>thresh) & (len(set(clusters))>12)):
            iter += 1
            result = np.where(sim_matrix1 == np.amax(sim_matrix1))
            if result[0][0]<result[1][0]:  
                i1 = result[0][0]
                i2 = result[1][0]
            else:
                i1 = result[1][0]
                i2 = result[0][0]

            #print('*****iteration',iter,'***********')

            #print('clusters:',i1,i2,'labels:',l[i1],l[i2],'distance:', np.amax(sim_matrix1))

            clusters[clusters==i2]=i1
            clusters[clusters>i2] -= 1

            cluster_len = len(set(clusters))
            l =np.zeros((cluster_len))
            purity = np.zeros((cluster_len))
            for k in range(cluster_len):
                part = [indx for indx, n in enumerate(clusters) if n==k]
                vote = []
                vote = np.zeros(max(Y)+1)
                for p in part:
                    vote[Y[p]-1] += 1
                purity[k] = vote.max()/sum(vote)
                l[k] = np.argmax(vote)

            #print('purity:',np.mean(purity), 'cluster#:', len(set(clusters)))
            #print('nmi:', normalized_mutual_info_score(clusters,Y))


            #print('*********************************')

            for i in range(len(w_density1)):
                w_density1[i][i1] = w_density1[i][i1] + w_density1[i][i2]
                w_density1[i1][i] = w_density1[i][i1]
                density1[i][i1] = density1[i][i1] + density1[i][i2]
                density1[i1][i] = density1[i][i1]

            for j in range(len(sim_matrix1)):
                if i1!=j:
                    sim_matrix1[i1][j]= similarity(w_density1,density1,i1,j)
                    #print( sim_matrix1[i1][j],i1,j)
                else:
                    sim_matrix1[i1][j]=0
                    sim_matrix1[j][i1]=0

            w_density1=np.delete(w_density1,i2,0)
            w_density1=np.delete(w_density1,i2,1)

            density1=np.delete(density1,i2,0)
            density1=np.delete(density1,i2,1)

            sim_matrix1=np.delete(sim_matrix1,i2,0)
            sim_matrix1=np.delete(sim_matrix1,i2,1)

        #print('purity:',np.mean(purity), 'cluster#:', len(set(clusters)))
        #print('nmi:', normalized_mutual_info_score(clusters,Y))

        nmi.append(normalized_mutual_info_score(clusters,Y))

    avg_nmi = np.mean(np.array(nmi),axis=0)
    std_nmi = np.std(np.array(nmi),axis=0)
    print('**********'+unit+'*************')
    print(avg_nmi)

    dbfile = open('datasets/pamap2/'+unit+'_avg_nmi_score_cci', 'ab') 
    pkl.dump(avg_nmi,dbfile)
    dbfile.close()

    dbfile = open('datasets/pamap2/'+unit+'_std_nmi_score_cci', 'ab') 
    pkl.dump(std_nmi,dbfile)
    dbfile.close()

In [246]:
activity_size = 12
clusters = np.array(parts)

sim_matrix1 = np.array(sim_matrix)
nonZ_sim = sim_matrix1[np.nonzero(sim_matrix1)]

w_density1 = np.array(w_density)
density1 = np.array(density)

threshold = 0# np.nanmedian(nonZ_sim)
itr =0
curr_size = len(set(clusters))
prev_size = 0
while True:
    
    if ((curr_size == prev_size) | (curr_size <= activity_size)):
        print('BREAK1',prev_size,curr_size,activity_size)
        break
    else: 
        prev_size = curr_size
        
    cluster_iter = 0
    while((cluster_iter < curr_size) & (curr_size > activity_size)):
        
        if not np.max(sim_matrix1[cluster_iter][:]):
            #print('BREAK2','max=',np.max(sim_matrix1[cluster_iter][:]))
            cluster_iter += 1
            continue
   
        max_indx = np.nanargmax(sim_matrix1[cluster_iter][:])
    
        if sim_matrix1[cluster_iter][max_indx]>threshold:
            if max_indx>cluster_iter:  
                i1 = cluster_iter
                i2 = max_indx
            else:
                i1 = max_indx
                i2 = cluster_iter
            
            itr +=1
            
            l =np.zeros((curr_size))
            cluster_size = np.zeros((curr_size))
            for k in range(curr_size):
                part = [indx for indx, n in enumerate(clusters) if n==k]
                cluster_size[k] = len(part)
                vote = np.zeros(len(set(Y)))
                for p in part:
                    vote[Y[p]-1] += 1
                l[k] = np.argmax(vote)
                
            print('*****iteration',itr,'***********')

            print('clusters:',cluster_iter,max_indx,'labels:',l[cluster_iter],l[max_indx],'distance:', 
                 sim_matrix1[cluster_iter][max_indx])
            print('density:',density1[cluster_iter][cluster_iter]/cluster_size[cluster_iter],
                  density1[max_indx][max_indx]/cluster_size[max_indx],w_density1[cluster_iter][cluster_iter]/cluster_size[cluster_iter],
                  w_density1[max_indx][max_indx]/cluster_size[max_indx])
            clusters[clusters==i2]=i1
            clusters[clusters>i2] -= 1
            
            curr_size = len(set(clusters))
            purity = np.zeros((curr_size))
            for k in range(curr_size):
                part = [indx for indx, n in enumerate(clusters) if n==k]
                vote = np.zeros(len(set(Y)))
                for p in part:
                    vote[Y[p]-1] += 1
                purity[k] = vote.max()/sum(vote)

            print('purity:',np.mean(purity), 'cluster#:', len(set(clusters)))
            print('nmi:', normalized_mutual_info_score(clusters,Y,average_method='arithmetic'))

            print('*********************************')

            for i in range(len(w_density1)):
                w_density1[i1][i] += w_density1[i2][i]
                w_density1[i][i1] += w_density1[i][i2]
                density1[i1][i] += density1[i2][i]
                density1[i][i1] += density1[i][i2]

            for j in range(len(sim_matrix1)):
                if i1!=j:
                    sim_matrix1[i1][j]= similarity(w_density1,density1,i1,j)
                else:
                    sim_matrix1[i1][j]=0
                    sim_matrix1[j][i1]=0

            w_density1=np.delete(w_density1,i2,0)
            w_density1=np.delete(w_density1,i2,1)

            density1=np.delete(density1,i2,0)
            density1=np.delete(density1,i2,1)

            sim_matrix1=np.delete(sim_matrix1,i2,0)
            sim_matrix1=np.delete(sim_matrix1,i2,1)
            
        cluster_iter +=1

*****iteration 1 ***********
clusters: 0 53 labels: 11.0 11.0 distance: 0.18294224416453492
density: 16.20689655172414 7.111111111111111 133.74336392915149 86.98265320977755
purity: 0.9041564102200094 cluster#: 62
nmi: 0.672014082545793
*********************************
*****iteration 2 ***********
clusters: 1 4 labels: 7.0 7.0 distance: 1.1472318273705346
density: 11.176470588235293 10.0 94.36369792452714 81.98192373876132
purity: 0.9075727428099978 cluster#: 61
nmi: 0.6727774460555483
*********************************
*****iteration 3 ***********
clusters: 2 3 labels: 1.0 1.0 distance: 0.014087944816648369
density: 15.483870967741936 13.0 189.03955346173586 114.14765131014352
purity: 0.9070453604189227 cluster#: 60
nmi: 0.6750811614171336
*********************************
*****iteration 4 ***********
clusters: 3 12 labels: 4.0 4.0 distance: 0.911271545079276
density: 11.052631578947368 12.842105263157896 88.22039141769905 91.14480086614545
purity: 0.9054698580531418 cluster#: 59
nmi:

*****iteration 33 ***********
clusters: 10 24 labels: 5.0 5.0 distance: 0.0006421371916266977
density: 10.62962962962963 16.642857142857142 83.95619825313258 129.41166224786477
purity: 0.8725359063152742 cluster#: 30
nmi: 0.7116946928565988
*********************************
*****iteration 34 ***********
clusters: 11 13 labels: 2.0 2.0 distance: 0.017943970206782837
density: 4.558139534883721 17.565217391304348 31.92319143677915 163.8164198566353
purity: 0.8681405927399388 cluster#: 29
nmi: 0.7159285513440584
*********************************
*****iteration 35 ***********
clusters: 12 1 labels: 7.0 7.0 distance: 0.18257856396731734
density: 10.142857142857142 12.785714285714286 62.4494538678009 79.13222395724594
purity: 0.8757612601677257 cluster#: 28
nmi: 0.7197342045324172
*********************************
*****iteration 36 ***********
clusters: 13 18 labels: 0.0 0.0 distance: 0.0023458500657986816
density: 10.418604651162791 3.0 88.43758437006043 34.78380863715375
purity: 0.871159825

In [ ]:
pos = nx.spring_layout(G)  # compute graph layout
fig= plt.figure(figsize=(20, 20))  # image is 8 x 8 inches
plt.axis('off')
nx.draw_networkx_nodes(G, pos, node_size=10, cmap=plt.cm.RdYlBu, node_color=clusters)
nx.draw_networkx_edges(G, pos, alpha=0.3)
plt.show(G)
fig.savefig('net.pdf')

In [ ]:
options = {
    'node_color': 'black',
    'node_size': 50,
    'line_color': 'grey',
    'linewidths': 0,
    'width': 0.1,
}
nx.draw(G, **options)
plt.show()